In [1]:
import os

In [2]:
%pwd

'd:\\Machine-learning-project-with-Mlflow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Machine-learning-project-with-Mlflow'

In [5]:
from dotenv import load_dotenv
import dagshub

load_dotenv()

dagshub.init(repo_owner='bennedictbett', 
             repo_name='Machine-learning-project-with-Mlflow', 
             mlflow=True)

Accessing as bennedictbett

Initialized MLflow to track repo "bennedictbett/Machine-learning-project-with-Mlflow"

Repository bennedictbett/Machine-learning-project-with-Mlflow initialized!

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_path: Path
    target_column: str
    mlflow_url: str

In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json


In [8]:
class ConfigurationManager:
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])  

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_path=config.metric_file_path,
            target_column=schema.name,
            mlflow_url=config.mlflow_url
        )

        return model_evaluation_config

In [9]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [ ]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
        # Load environment variables
        load_dotenv()
        
        # Initialize DagsHub
        dagshub.init(repo_owner='bennedictbett', 
                     repo_name='Machine-learning-project-with-Mlflow', 
                     mlflow=True)
        
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        x_test = test_data.drop(columns=[self.config.target_column], axis=1)
        y_test = test_data[self.config.target_column]

        mlflow.set_tracking_uri(self.config.mlflow_url)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(x_test)

            (rmse, mae, r2) = self.eval_metrics(y_test, predicted_qualities)

            # Save metrics locally
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_path), data=scores)

            # Log parameters to MLflow
            mlflow.log_params(self.config.all_params)

            
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)

            # Log model
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetWineModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [12]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-12-31 13:52:19,138 : INFO : common : YAML file 'config\config.yaml' read successfully.]
[2025-12-31 13:52:19,144 : INFO : common : YAML file 'params.yaml' read successfully.]
[2025-12-31 13:52:19,151 : INFO : common : YAML file 'config\schema.yaml' read successfully.]
[2025-12-31 13:52:19,154 : INFO : common : Created directory at: artifacts_root]
[2025-12-31 13:52:19,156 : INFO : common : Created directory at: artifacts/model_evaluation]
[2025-12-31 13:52:20,770 : INFO : _client : HTTP Request: GET https://dagshub.com/api/v1/repos/bennedictbett/Machine-learning-project-with-Mlflow "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "bennedictbett/Machine-learning-project-with-Mlflow"

[2025-12-31 13:52:20,793 : INFO : helpers : Initialized MLflow to track repo "bennedictbett/Machine-learning-project-with-Mlflow"]


Repository bennedictbett/Machine-learning-project-with-Mlflow initialized!

[2025-12-31 13:52:20,804 : INFO : helpers : Repository bennedictbett/Machine-learning-project-with-Mlflow initialized!]
[2025-12-31 13:52:26,313 : INFO : common : JSON file saved at: artifacts\model_evaluation\metrics.json]


2025/12/31 13:52:33 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Successfully registered model 'ElasticNetWineModel'.
2025/12/31 13:53:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNetWineModel, version 1
Created version '1' of model 'ElasticNetWineModel'.


🏃 View run sincere-bird-446 at: https://dagshub.com/bennedictbett/Machine-learning-project-with-Mlflow.mlflow/#/experiments/0/runs/e90aea9b57c44dee812812d314b26d85
🧪 View experiment at: https://dagshub.com/bennedictbett/Machine-learning-project-with-Mlflow.mlflow/#/experiments/0
